In [1]:
import cv2
import numpy as np
import pandas as pd

In [2]:
def extract_features(image):
    import cv2
    import numpy as np

    # Convert to grayscale for processing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(
        blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, 11, 2
    )

    # Remove small noise with morphological operations
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(
        thresh, cv2.MORPH_OPEN, kernel, iterations=2
    )

    # Find contours
    contours, _ = cv2.findContours(
        opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    features_list = []

    for cnt in contours:
        # Filter by area (ignore very small particles)
        area = cv2.contourArea(cnt)
        if area > 20:
            # Compute perimeter
            perimeter = cv2.arcLength(cnt, True)

            # Compactness
            compactness = (perimeter ** 2) / area if area != 0 else 0

            # Bounding rectangle
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = float(w) / h if h != 0 else 0

            # Fit ellipse if possible
            if len(cnt) >= 5:
                ellipse = cv2.fitEllipse(cnt)
                _, axes, _ = ellipse
                major_axis = max(axes)
                minor_axis = min(axes)
                ellipse_axis_ratio = minor_axis / major_axis if major_axis != 0 else 0
            else:
                ellipse_axis_ratio = 0

            # Create a mask for the contour
            mask = np.zeros(image.shape[:2], dtype="uint8")
            cv2.drawContours(mask, [cnt], -1, 255, -1)

            # Calculate mean color within the contour mask
            mean_val = cv2.mean(image, mask=mask)
            mean_b, mean_g, mean_r = mean_val[:3]

            # Convert BGR to HSV and get mean hue
            hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            mean_hsv = cv2.mean(hsv_image, mask=mask)
            mean_hue = mean_hsv[0]

            # Collect features
            features = {
                'Area': area,
                'Perimeter': perimeter,
                'Compactness': compactness,
                'Aspect_Ratio': aspect_ratio,
                'Ellipse_Axis_Ratio': ellipse_axis_ratio,
                'Mean_R': mean_r,
                'Mean_G': mean_g,
                'Mean_B': mean_b,
                'Mean_Hue': mean_hue,
                'Contour': cnt,  # Keep contour for visualization
                'Bounding_Rect': (x, y, w, h)  # For labeling
            }
            features_list.append(features)

    return features_list

In [3]:
def classify_particle(features, params):
    # Unpack features
    area = features['Area']
    perimeter = features['Perimeter']
    compactness = features['Compactness']
    aspect_ratio = features['Aspect_Ratio']
    ellipse_axis_ratio = features['Ellipse_Axis_Ratio']
    mean_r = features['Mean_R']
    mean_g = features['Mean_G']
    mean_b = features['Mean_B']
    mean_hue = features['Mean_Hue']

    # Calculate mean brightness
    mean_brightness = (mean_r + mean_g + mean_b) / 3

    # Unpack parameters
    compactness_threshold = params['compactness_threshold']
    ellipse_axis_ratio_threshold = params['ellipse_axis_ratio_threshold']
    mean_hue_pellet_range = params['mean_hue_pellet_range']
    mean_brightness_fragment_threshold = params['mean_brightness_fragment_threshold']
    mean_brightness_tar_threshold = params['mean_brightness_tar_threshold']

    # Classification rules
    if (compactness >= compactness_threshold and
        ellipse_axis_ratio >= ellipse_axis_ratio_threshold and
        mean_hue >= mean_hue_pellet_range[0] and
        mean_hue <= mean_hue_pellet_range[1]):
        return 'Pellet'
    elif mean_brightness <= mean_brightness_tar_threshold:
        return 'Tar'
    elif mean_brightness >= mean_brightness_fragment_threshold:
        return 'Fragment'
    else:
        return 'Unknown'

In [4]:
params = {
    'compactness_threshold': 15.0,
    'ellipse_axis_ratio_threshold': 0.8,
    'mean_hue_pellet_range': (15, 45),  # Hue range for pellets
    'mean_brightness_fragment_threshold': 100,  # Bright fragments
    'mean_brightness_tar_threshold': 80  # Dark tar particles
}

In [5]:
def process_and_classify_image(image_file, params):
    import cv2
    import pandas as pd

    # Load the image
    image = cv2.imread(image_file)
    if image is None:
        print(f"Error loading image {image_file}")
        return None

    features_list = extract_features(image)

    # Prepare results list
    results = []
    for i, features in enumerate(features_list):
        predicted_label = classify_particle(features, params)
        features['Predicted_Label'] = predicted_label
        features['Particle_Number'] = i + 1
        results.append(features)

        # Draw the contour and label on the image
        cv2.drawContours(image, [features['Contour']], -1, (0, 255, 0), 2)
        x, y, w, h = features['Bounding_Rect']
        cv2.putText(image, predicted_label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Convert results to DataFrame
    df_results = pd.DataFrame(results)

    # Show the image with classifications
    cv2.imshow('Classified Particles', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return df_results

In [ ]:
# Path to the new image
test_image_file = 'TAR.png'

# Process and classify the new image
df_test_results = process_and_classify_image(test_image_file, params)

# Display the classification results
if df_test_results is not None:
    print(df_test_results[['Particle_Number', 'Predicted_Label']])
